In [2]:
import glob
import os
import re
import zipfile
import pandas as pd
import requests
from ete3 import NCBITaxa

region2_Taeniopygia_guttata = [['chr1A', 25426409, 25426646,'taeGut1_dna-range-chr1A:25426409-25426646-kmviz-631a704a-45ee-4268-864f-2b111e96984c.zip']]

def create_region_list(file_search_string):
    region_list = []
    for file in glob.glob(file_search_string):
        if file.endswith(".zip"):
            fileparts = re.split(r':|-', os.path.basename(file))
            region_list.append([fileparts[0], fileparts[1], fileparts[2], file])
    return region_list

def process_regions(region):
    return pd.concat([data(region[i][-1], 'chr1A', region[i][1], region[i][2]) for i in range(0,len(region))], ignore_index=True)

def data(file, chr, start, stop, prefix=''):
    with zipfile.ZipFile(glob.glob(os.path.join(prefix, file))[0], 'r') as z:
        tsv_file = [name for name in z.namelist() if name.endswith('.tsv')][0]
        with z.open(tsv_file) as f:
            df = pd.read_csv(f, sep="\t")
            df = df.assign(chr=chr, start=start, stop=stop)
            return df

def print_files_assay_type(region, file, color):
    if not os.path.isdir(os.path.dirname(file)):
        os.mkdir(os.path.dirname(file))
    keys = []
    for i in range(0, len(region)):
        for t in region[i][-1]['assay_type'].unique():
            keys.append(t)
    keys = list(set(keys))
    keys = dict(list(zip(keys, range(len(keys)))))
    with open(f"{file}.bed", 'w') as output:
        output.write(f'track name="loganSearch assay type"  visibility=2 itemRgb="On"')
        for i in range(0, len(region)):
            create_bedrows(region[i], keys, output, color, 'assay_type')

zebra_finch = create_region_list("./logan-result/NC_044212.2:*.zip")
zebra_finch = process_regions(zebra_finch)
colors_8 = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]

colors_20 = [
    "#E69F00",  # orange
    "#56B4E9",  # light blue
    "#009E73",  # green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # reddish orange
    "#CC79A7",  # pink
    "#999999",  # gray
    "#ADFF2F",  # green-yellow
    "#FF69B4",  # hot pink
    "#87CEFA",  # sky blue
    "#6A5ACD",  # slate blue
    "#FFD700",  # gold
    "#40E0D0",  # turquoise
    "#DC143C",  # crimson
    "#8B4513",  # saddle brown
    "#00CED1",  # dark turquoise
    "#DA70D6",  # orchid
    "#228B22",  # forest green
    "#8A2BE2",  # blue violet
]

colors_20_rgb = [
    "230,159,0",    # orange
    "86,180,233",   # light blue
    "0,158,115",    # green
    "240,228,66",   # yellow
    "0,114,178",    # blue
    "213,94,0",     # reddish orange
    "204,121,167",  # pink
    "153,153,153",  # gray
    "173,255,47",   # green-yellow
    "255,105,180",  # hot pink
    "135,206,250",  # sky blue
    "106,90,205",   # slate blue
    "255,215,0",    # gold
    "64,224,208",   # turquoise
    "220,20,60",    # crimson
    "139,69,19",    # saddle brown
    "0,206,209",    # dark turquoise
    "218,112,214",  # orchid
    "34,139,34",    # forest green
    "138,43,226",   # blue violet
]

/tmp/ipykernel_37390/3181955274.py:26: DtypeWarning: Columns (16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep="\t")


In [20]:
desired_ranks = ["domain", "superkingdom", "kingdom", "phylum", "class", 
                             "order", "family", "genus"]

def getTaxInformation(species):
    ncbi = NCBITaxa()
    #ncbi.update_taxonomy_database()
    tax_information = ncbi.get_name_translator([key for key in species])
    for key in species:
        if key not in tax_information:
            genus = key.split(" ")[0]
            genus_search = ncbi.get_name_translator([genus])
            if genus in genus_search:
                tax_information[key] = genus_search[genus]
            else:
                raise Exception(f"Couldn't locate {key} or {genus}, {genus_search}")
    taxInfo = {}
    for key in tax_information:
        taxInfo[key] = {'species': key, 'tax_id': tax_information[key][0]}
        
        lineage = ncbi.get_lineage(tax_information[key][0])
        names = ncbi.get_taxid_translator(lineage)  # dict: taxid -> name
        ranks = ncbi.get_rank(lineage)  # dict: taxid -> rank

        classification = {}
        for tax in lineage:
            rank = ranks.get(tax)
            name = names.get(tax)
            if rank in desired_ranks:
                classification[rank] = name

        # Convert to printable structure
        ordered_classification = []
        for r in desired_ranks:            
            taxInfo[key][r] = classification.get(r, 'NA')
            #if taxInfo[key][r] == 'N/A' and r != 'superkingdom':
            #    print(key, r)
    return taxInfo

def print_files_organism(region, file, color):
    if not os.path.isdir(os.path.dirname(file)):
        os.mkdir(os.path.dirname(file))
    keys = []
    for i in range(0, len(region)):
        for t in region[i][-1]['organism'].unique():
            keys.append(t)
    keys = list(set(keys))
    counter = 0
    temp = []
    for key in keys:
        temp.append((key, counter))
        counter+=1
        if counter >= 20:
            counter = 0
    keys = dict(temp)
    with open(f"{file}.bed", 'w') as output:
        output.write(f'track name="loganSearch organism"  visibility=2 itemRgb="On"')
        for i in range(0, len(region)):
            create_bedrows(region[i], keys, output, color, 'organism')
    #         output.write("\n")
    return keys

def create_hubtrack_and_beggraphs(df, organism, genome, group_key, group_values, color_list, base_url, composite=False):
    folder = f"{organism.replace(" ", "")}/{group_key}"
    os.makedirs(folder, exist_ok=True)

    # Create hub file
    hub_file = f"{folder}/hub.txt"
    genome_file = f"{folder}/genomes.txt"
    track_file = f"{folder}/trackDb.txt"
    composite_track_string = ""
    if composite:
        composite_track_string = "Composite"
        genome_file = f"{folder}/genomes.{composite_track_string}.txt"
        track_file = f"{folder}/trackDb.{composite_track_string}.txt"
        hub_file = f"{folder}/hub.{composite_track_string}.txt"
    
    with open(f"{folder}/hub.txt", "w") as hub_out:
        hub_out.write(f"hub {organism.replace(' ', '')}LoganSearch{group_key}{composite_track_string}")
        hub_out.write(f"\nshortLabel {organism}: {group_key}")
        hub_out.write(f"\nlongLabel Logan search data for {organism} grouped by {group_key}")
        hub_out.write(f"\ngenomesFile {os.path.basename(genome_file)}")
        hub_out.write("\nemail pks5769@psu.edu")
        hub_out.write(f"\ndescriptionUrl {base_url}/{folder}/description.html")
    
    with open(genome_file, "w") as genomes_out:
        genomes_out.write(f"genome {genome}")
        genomes_out.write(f"\ntrackDb {os.path.basename(track_file)}")

    with open(f"{folder}/description.html", "w") as description_out:
        description_out.write(f"<html><head><title>{organism}: {group_key}</title></head><body>{organism}</body></html>")
 
    with open(track_file, "w") as trackDB_output:
        trackDB_output.write(f"track {organism.replace(' ', '')}LoganSearchResult")      
        trackDB_output.write("\ntype bigWig")
        if composite:
            trackDB_output.write("\ncompositeTrack on")
            trackDB_output.write(f"\nshortLabel {organism}: {group_key} : {composite_track_string}")
            trackDB_output.write(f"\nlongLabel {organism} grouped by {group_key}, {composite_track_string}")
        else:
            trackDB_output.write("\ncontainer multiWig")
            trackDB_output.write(f"\nshortLabel {organism}: {group_key}")
            trackDB_output.write(f"\nlongLabel {organism} grouped by {group_key}")
        trackDB_output.write("\nautoScale on")
        trackDB_output.write("\naggregate transparentOverlay")
        trackDB_output.write("\nshowSubtrackColorOnUi on")
        trackDB_output.write("\nmaxHeightPixels 500:100:8")
        for v, c_i in group_values.items():
            trackDB_output.write(f"\n\t\n\ttrack {v}")
            trackDB_output.write("\n\tautoScale on")
            trackDB_output.write(f"\n\tshortLabel {v}")
            trackDB_output.write(f"\n\tlongLabel {v}")
            trackDB_output.write(f"\n\tbigDataUrl {base_url}{folder}/bw/{v}.bw")
            trackDB_output.write(f"\n\ttype bigWig")
            trackDB_output.write(f"\n\tparent {organism.replace(' ', '')}LoganSearchResult on")
            trackDB_output.write(f"\n\tcolor {color_list[c_i]}")
            trackDB_output.write(f"\n\tdescriptionUrl https://raw.githubusercontent.com/Smeds/bigWigTest/refs/heads/main/description.html")

            os.makedirs(f"{folder}/bedGraph", exist_ok=True)
            with open(f"{folder}/bedGraph/{v}.bedGraph", 'w') as output:
                output.write(f'track type=bedGraph  name="loganSearch {group_key} {v}"  visibility=2 itemRgb="On"\n')
                for index, row in df[df[group_key] == v].groupby(['chr', 'start', 'stop', group_key]).size().reset_index(name='count').iterrows():
                    output.write(f"{row['chr']}\t{row['start']}\t{row['stop']}\t{row['count']}\n")
    
def create_bedrows(region, keys, output, color, group_by):
    for i, r in region[-1].iterrows():
        output.write(f"\n{region[0]}"
                     f"\t{region[1]}"
                     f"\t{region[2]}"
                     f"\t{r['ID']}"
                     f"\t1"
                     f"\t."
                     f"\t{region[1]}"
                     f"\t{region[2]}"
                     f"\t{color[keys[r[color_by]]]}")

def print_files_group_by_classification(df, genome, organism, color, group_key, base_url, composite):
    organism_list = []
    group_values = []
    #for i in range(0, len(region)):
    #    for o in region[i][-1]['organism'].unique():
    #        organism.append(o)
    organism_list = list(set(df['organism'].unique()))
    group_values = list(set(group_values))
    counter = 0
    temp = []   
    
    tax_information = getTaxInformation(organism_list)

    
    class_list = []
    for index, row in df.iterrows():
        class_list.append([tax_information[row['organism']][d] for d in desired_ranks])
    df[desired_ranks] = pd.DataFrame(class_list)
    
    group_values = df[group_key].unique()
    
    for g in group_values:
        temp.append((g, counter))
        counter+=1
        if counter >= 20:
            counter = 0
    group_values = dict(temp)
    #def create_bedGraph_assay_type(df, organism, genome, group_key, group_values, color_list):
    create_hubtrack_and_beggraphs(df, organism, genome, group_key, group_values, color, base_url, composite)
            #create_bedrows(region[i], group_values, output, color, group_key)
            #return

#species = print_files_organism(zebra_finch, "zebrafinch/region2_Taeniopygia_guttata_organism", color=colors_20_rgb)

print_files_group_by_classification(zebra_finch, organism="Zebra Finch", genome="GCF_003957565.2", color=colors_20_rgb, group_key='class', base_url="https://raw.githubusercontent.com/Smeds/hubtrack/refs/heads/main/", composite=False)
print_files_group_by_classification(zebra_finch, organism="Zebra Finch", genome="GCF_003957565.2", color=colors_20_rgb, group_key='class', base_url="https://raw.githubusercontent.com/Smeds/hubtrack/refs/heads/main/", composite=True)




trackDb.txt
trackDb.Composite.txt
